In [1]:
import os
import functions as fun
from geopy import distance
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from dotenv import load_dotenv
import pandas as pd

In [2]:
sub_CH = pd.read_csv("../Data/sub_CH.csv")


# Chicago

In [3]:
Chicago = 41.869276, -87.626694

In [4]:
load_dotenv()

True

In [5]:
token_foursquare = os.getenv("Token_foursquare")

- First we look for Starbucks close to the office and clean the results keeping only the columns that we are interested about

In [6]:
starbucks_CH = fun.get_results("starbucks", Chicago, 10, 1000, token_foursquare)
df_CH = fun.clean_results_first(starbucks_CH, "Coffee")

- We look for preschools or kindergarten around the office and clean the results keeping only the columns that we are interested about

In [7]:
kinder_CH = fun.get_results("preschool", Chicago, 10, 3000, token_foursquare)
df_CH = fun.clean_results(kinder_CH, df_CH, "Preschool")

- We look for nightclubs around the office and clean the results keeping only the columns that we are interested about

In [8]:
club_CH = fun.get_results("night club", Chicago, 10, 3000, token_foursquare)
df_CH = fun.clean_results(club_CH, df_CH, "Club")

- We look for vegan restaurants around the office and clean the results keeping only the columns that we are interested about

In [9]:
vegan_CH = fun.get_results("vegan", Chicago, 10, 3000, token_foursquare)
df_CH = fun.clean_results(vegan_CH, df_CH, "Vegan restaurant")

- We look for dog hairdressers restaurants around the office and clean the results keeping only the columns that we are interested about

In [10]:
pet_CH = fun.get_results("pet grooming", Chicago, 5, 3000, token_foursquare)
df_CH = fun.clean_results(pet_CH, df_CH, "Pet hairdresser")

- We look for basketball arena. However, there are not major teams playing nowadays in Seattle. We pray for the return of the Supersonics

In [11]:
Basketball_CH = fun.get_results("United center", Chicago, 1, 5000, token_foursquare)
df_CH = fun.clean_results(Basketball_CH, df_CH, "Basketball arena")

- We look for the closest Airport to office and clean the results keeping only the columns that we are interested about

In [12]:
Airports_CH = fun.get_results_airport("Airport", Chicago, 1, 50000, token_foursquare)
df_CH = fun.clean_results(Airports_CH, df_CH, "Airport")

- We add the data of the new office that we have selected

In [13]:
df_CH = fun.add_company(df_CH, "118 S Holden Ct, Chicago, IL 60605", "Dearborn Park", 41.869276, -87.626694)

- We create a new row with the distance from the office to the diferent places

In [14]:
df_CH["Distance"] = df_CH.apply(fun.calc_distance_CH, axis = 1).round(2)

In [15]:
df_CH

,name,address,neighborhood,lat,lon,category,Distance
0,Company,"118 S Holden Ct, Chicago, IL 60605",Dearborn Park,41.869276,-87.626694,NaN,0.00
1,KinderCare Learning Center,"1501 S State St, Chicago, IL 60605",Film Row,41.862063,-87.627012,Preschool,801.59
2,Sonnets Academy,"229 S Peoria St (W Jackson Blvd), Chicago, IL ...",Duncans Addition,41.878239,-87.649239,Preschool,2119.86
3,Downtown Children Learning Pl,"400 N McClurg Ct, Chicago, IL 60611",Cityfront Center,41.889244,-87.617852,Preschool,2336.15
4,Mark T. Skinner School Pre-K,"225 S Aberdeen St, Chicago, IL 60607",Duncans Addition,41.878268,-87.654313,Preschool,2500.86
5,Lily Pad Nursery Preschool,"1025 W Van Buren St, Chicago, IL 60607",Duncans Addition,41.876364,-87.652857,Preschool,2310.22
6,Sprouts Academy Preschool Managed By Bright Ho...,"350 W Ontario St, Chicago, IL 60654",Killgubbin,41.893254,-87.637884,Preschool,2820.58
7,Bright Horizons at Cook County/City of Chicago...,"40 N Dearborn St (Washington), Chicago, IL 60602",Fort Dearborn Addition,41.882690,-87.629507,Preschool,1508.09
8,Bright Horizons at Lakeshore East,"360 E South Water St, Chicago, IL 60601",Cityfront Center,41.886588,-87.618185,Preschool,2048.48
9,Bright Horizons At River East,"325 E Grand Ave, Chicago, IL 60611",Cityfront Center,41.891552,-87.619217,Preschool,2550.87


In [16]:
CH_map = Map(location = Chicago, zoom_start = 12)

In [17]:
for index, row in df_CH.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["lat"], row["lon"]], "tooltip": row["name"]}
    
    #2. Icon
    if row["category"] == "Coffee":        
        icon = Icon (
            color="white",
            opacity = 0.6,
            prefix = "fa",
            icon="coffee",
            icon_color = "black"
        )
    elif row["category"] == "Preschool":
        icon = Icon (
            color="pink",
            opacity = 0.6,
            prefix = "fa",
            icon="graduation-cap",
            icon_color = "yellow"
        )
    elif row["category"] == "Club":
        icon = Icon (
            color="darkpurple",
            opacity = 0.6,
            prefix = "fa",
            icon="glass",
            icon_color = "white"
        )
    elif row["category"] == "Vegan restaurant":
        icon = Icon (
            color="green",
            opacity = 0.6,
            prefix = "fa",
            icon="cutlery",
            icon_color = "white"
        )
    elif row["category"] == "Basketball arena":
        icon = Icon (
            color="orange",
            opacity = 0.6,
            prefix = "fa",
            icon="futbol-o",
            icon_color = "white"
        )
    elif row["category"] == "Airport":
        icon = Icon (
            color="blue",
            opacity = 0.6,
            prefix = "fa",
            icon="plane",
            icon_color = "white"
            
        )
    elif row["category"] == "Pet hairdresser":
        icon = Icon (
            color="red",
            opacity = 0.6,
            prefix = "fa",
            icon="scissors",
            icon_color = "white"
            
        )
    else:
        row["category"] == "Company",
        icon = Icon (
        color="black",
        opacity = 0.9,
        prefix = "fa",
        icon = "briefcase",
        icon_color = "white",
        icon_size=(30, 30)
        )
            
    #3. Marker
    marker = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    marker.add_to(CH_map)

In [18]:
for index, row in sub_CH.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["Lat"], row["Long"]], "tooltip": row["name"]}
    
    #2. Icon       
    icon = Icon (
    color="white",
    opacity = 0.6,
    prefix = "fa",
    icon="building",
    icon_color = "black"
        )

            
    #3. Marker
    marker = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    marker.add_to(CH_map)

In [19]:
CH_map

In [20]:
CH_map.save('../images/Chicago_map.html')